In [1]:
# リスト 2.2.2 Janome サンプルコード
# ------------------------------------------------------------------------------------------------
# 目的:
#   ・純Python形態素解析器「Janome」を用いて、（1）分かち書き（wakati）と
#     （2）形態素トークン列（品詞等の素性を含む）の 2 パターンを確認する。
#
# 理論メモ（実装は変えず、背景のみ説明）:
#   ・Janome は MeCab と異なり C バインディング不要の純Python実装 → 導入容易だが速度は相対的に遅い。
#   ・辞書は IPADIC 互換フォーマット相当（Janome同梱）。そのため feature 列（品詞体系や列順）は
#     IPADIC の慣習（品詞, 品詞細分類1-3, 活用型, 活用形, 原形, 読み, 発音）に準じる。
#   ・Tokenizer(wakati=True) は「分かち書きモード」。`tokenize(text)` は表層形（str）の列を返す。
#     可視化としては文字列の join で `" ".join(...)` とするのが一般だが、ここでは元コードどおり list をそのまま出力。
#   ・Tokenizer()（通常モード）の `tokenize(text)` は Token オブジェクト列を返す。
#     Token の `__str__` は「表層形[TAB]feature列」を返すので、print(token) で人間可読な1行出力になる。
#   ・句読点「。」などの記号も1トークンとして出現（例：`記号,句点,*,*,*,*,*,*,*`）。
#     記号を残す/除くは下流処理（学習・検索・要約）に影響するため、仕様として方針を固定するのが望ましい。
#   ・Unicode 正規化（NFC/NFKC）は分割境界に影響しうる。再現性を重視するなら入力正規化の有無を運用規約に明記。
#   ・ユーザー辞書（janome.dic.userdic.UserDictionary）を使えば固有名詞等の OOV を低減できる。
#   ・スレッド/並列実行では、各スレッドごとに Tokenizer インスタンスを持つと安全（共有よりも衝突リスクが低い）。
#   ・速度最適化が必要なら、対象文を事前に文単位へ分割（バッチ化）し、イテレータで処理するなどでオーバーヘッドを抑える。
# ------------------------------------------------------------------------------------------------

# 解析対象文
text = "これは日本語の文章です。"  # 短い日本語文。末尾の句点「。」も1トークンとして扱われる

# 利用パターン1 対象文書を分かち書きにする
from janome.tokenizer import Tokenizer

t1 = Tokenizer(
    wakati=True
)  # 分かち書きモード。表層形のみのトークン列（list[str]）を返す

print(
    t1.tokenize(text)
)  # [ 'これ', 'は', '日本', '語', 'の', '文章', 'です', '。' ] のような配列（辞書/文により変動）
print()

# 利用パターン2 単語毎に分析結果を全部表示する
t2 = Tokenizer()  # 通常モード。Token オブジェクト列を返す

for token in t2.tokenize(text):
    # Token.__str__() は「表層形 + タブ + feature（カンマ区切り）」を返す。
    # 例: これ  代名詞,*,*,*,*,*,これ,コレ,コレ
    #     は    助詞,係助詞,*,*,*,*,は,ハ,ワ
    #     日本  名詞,固有名詞,地域,国,*,*,日本,ニホン,ニッポン
    #     語    名詞,一般,*,*,*,*,語,ゴ,ゴ
    #     の    助詞,連体化,*,*,*,*,の,ノ,ノ
    #     文章  名詞,一般,*,*,*,*,文章,ブンショウ,ブンショー
    #     です  助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
    #     。    記号,句点,*,*,*,*,*,*,*
    print(token)

<generator object Tokenizer.__tokenize_stream at 0x110ea2dc0>

これ	名詞,代名詞,一般,*,*,*,これ,コレ,コレ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
日本語	名詞,一般,*,*,*,*,日本語,ニホンゴ,ニホンゴ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
文章	名詞,一般,*,*,*,*,文章,ブンショウ,ブンショー
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。
